In [1]:
import pandas as pd
import numpy as np
from nltk.util import ngrams
np.random.seed(1377)

# HW3:  Классификация имен
В этом домашнем задании мы рассмотрим задачу бинарной классификации. Пусть дано два списка имен:  мужские и женские имена.  Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.
### Выполнили:  

* Булгаков Дмитрий

* Тефикова Алие

### Группа ИАД-2

## 0. Loading data

In [2]:
male_names = pd.read_csv('data/male.txt', header=-1, names=['name'], encoding = 'latin1')
female_names = pd.read_csv('data/female.txt', header=-1, names=['name'], encoding = 'latin1')

In [3]:
male_names_number = len(male_names)
female_names_number = len(female_names)
print('Количество мужских имен: ', male_names_number)
print('Количество женских имен: ', female_names_number)

Количество мужских имен:  2943
Количество женских имен:  5001


## 1. Filtering data

Предварительная обработка данных:<br>
* удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими одновременно), если такие есть.<br>
* создайте тестовое множество по следующему принципу: 20% от общего количества имен на каждую букву (т.е. 20% от имен на букву А, 20% имен на букву B, и.т.д.)

### 1.1 Looking for same names in both male and female df

In [4]:
same_names = female_names.merge(male_names, on=['name'], how='inner')
print('Количество совпадающих имен: ', len(same_names))
print('Список имен, являющихся одновременно и мужскими, и женскими: (первые 10)')
same_names.head(n = 10)

Количество совпадающих имен:  365
Список имен, являющихся одновременно и мужскими, и женскими: (первые 10)


,name
0,Abbey
1,Abbie
2,Abby
3,Addie
4,Adrian
5,Adrien
6,Ajay
7,Alex
8,Alexis
9,Alfie


### 1.2 And deleting duplicated names

In [5]:
remove_criterion = lambda row: row['name'] not in same_names['name'].values
male_names = male_names[male_names.apply(remove_criterion, axis=1)]
female_names = female_names[female_names.apply(remove_criterion, axis=1)]

In [6]:
print('Количество мужских имен после удаления дубликатов: ', len(male_names))
print('Количество женских имен после удаления дубликатов: ', len(female_names))
print('Все ок? ', (male_names_number - len(same_names) == len(male_names)) & 
      (female_names_number - len(same_names) == len(female_names)))

Количество мужских имен после удаления дубликатов:  2578
Количество женских имен после удаления дубликатов:  4636
Все ок?  True


### 1.3 Creating test sample (20% of dataset)

In [7]:
def getListOfFirstLetters(dataframe, field):
    letters = dataframe[field].astype(str).str[0]
    letters = np.unique(letters.tolist())
    return letters

In [8]:
def getStringsByFirstLetter(dataframe, field, letter):
    return dataframe[dataframe[field].str.startswith(letter)][field].tolist()

In [9]:
def getRandomSublistFromList(source_list, number_of_elements):
    return list(map(lambda _: np.random.choice(source_list), range(number_of_elements)))

In [10]:
def parseYArray(dataframe, array, field, field2):
    ans = []
    for value in array:
        ans.append(dataframe.loc[dataframe[field] == value][field2].tolist()[0])
    return ans

In [11]:
def createTrainTestSample(dataframe, field, field2, percentage=0.2):
    first_letters = getListOfFirstLetters(dataframe, field)
    resulting_train_sample = []
    resulting_test_sample = []
    train_y = []
    test_y = []
    for letter in first_letters:
        values_list = getStringsByFirstLetter(dataframe, field, letter)
        number_of_values_to_take = int(np.floor(len(values_list) * percentage))
        test_sample_for_letter = getRandomSublistFromList(values_list, number_of_values_to_take)
        train_sample_for_letter = [x for x in values_list if x not in test_sample_for_letter]
        test_y.append(parseYArray(dataframe, test_sample_for_letter, field, field2))
        train_y.append(parseYArray(dataframe, train_sample_for_letter, field, field2))
        resulting_test_sample.append(test_sample_for_letter)
        resulting_train_sample.append(train_sample_for_letter)
    return resulting_train_sample, resulting_test_sample, train_y, test_y

In [12]:
all_names = pd.concat([male_names, female_names], axis=0, ignore_index=True)
all_names['gender'] = np.concatenate((np.ones(len(male_names)), np.zeros(len(female_names))), axis=0).astype(int)
print('Общее количество имен: ', len(all_names))
print('Все ок? ', len(all_names) == len(male_names) + len(female_names))

Общее количество имен:  7214
Все ок?  True


In [ ]:
# getting list of fisrt letters of names
lettes_list = getListOfFirstLetters(all_names, 'name')
print('Список первых букв имен, присутствующих в датасете:')
lettes_list

Список первых букв имен, присутствующих в датасете:


array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], 
      dtype='<U1')

In [ ]:
X_train, X_test, y_train, y_test = createTrainTestSample(all_names, 'name', 'gender') 

## 2. Fitting Naive Bayes classifier

Используйте метод наивного Байеса для классификации имен: в качестве признаков используйте символьные n-граммы. <br>Сравните результаты, получаемые при разных n= 2, 3, 4 по F-мере и аккуратности. В каких случаях метод ошибается? <br>Для генерации n-грамм используйте <br><b>from nltk.util import ngrams.<b>